In [1]:
from sympy import *
import math
import numpy as np

### 18.1

In [2]:
A1=10
A2=6
dT=0
L1=50
L2=75
E=20*1e6
alp=11*1e-6
P1=P3=0
P2=15_000

In [3]:
def ElementStiffness(A,E,L):
    return A*E/L*np.matrix([[1,-1],[-1,1]])

In [4]:
def ElementThermalForce(A,E,alp,dT):
    return np.array([-A*E*alp*dT,
                       A*E*alp*dT])

In [5]:
def FEA_u(inp, bc_node, bc_val, P):
    e=len(inp)
    n=e+1
    
    elem=[]    
    for i in range(e):
        H=[i,i+1]
        elem.append(H)
    elem=np.array(elem)

    K=np.zeros(shape=(n, n))
    
    F=np.zeros(shape=(n,))
    
    for i in range(e):
        node1 = elem[i,0]
        node2 = elem[i,1]
        A, E, L, alp, dT = inp[i]
        k=ElementStiffness(A,E,L)
        K[node1:node2+1,node1:node2+1] = K[node1:node2+1,node1:node2+1] + k
        
        f = ElementThermalForce(A,E,alp,dT)
        F[node1:node2+1] = F[node1:node2+1] + f

    F+=P   
    bc=np.array([bc_node, bc_val]).T

    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))


    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    s=np.linalg.solve(K[fdof,:][:,fdof],(F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T)
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
#     Q=np.dot(K,d).T-F    
        
    return d

In [6]:
inp = [[A1, E, L1, alp, dT], [A2, E, L2, alp, dT]]

In [7]:
FEA_u(inp, [0,2], [0,0], np.array([P1,P2,P3]))

array([0.        , 0.00267857, 0.        ])

### 18.3

In [8]:
A=5
dT=10
L1=50
L2=50
L3=50
E=20*1e6
alp=11*1e-6
P2=P3=10_000
P1=P4=0

In [9]:
inp = [[A, E, L1, alp, dT], [A, E, L2, alp, dT], [A, E, L3, alp, dT]]

In [10]:
FEA_u(inp, [0,3], [0,0], np.array([P1,P2,P3,P3]))

array([0.   , 0.005, 0.005, 0.   ])